In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as  np
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
import statistics

In [3]:
cuda_available = torch.cuda.is_available()
print(cuda_available)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

True


In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [27]:
#load data (channel = subseq)

def load_file(file, flabel,subseq = 10, version = 1):
    #print(file)
    
    
    
    f = open(file, 'r')
    Lines = f.readlines()
    t = 1
    data = []
    for line in Lines:
        t += 1
        line = line.split()[0:75] #75=25*3

        data += line
    
    fl = open(flabel, 'r')
    label = fl.readlines()[0]
    #print (len(label))
    
    
    #data
    samples=len(label)
    data_len= samples * subseq * 75
    data = data[0:data_len]
    data = [float(i) for i in data] 
    data = np.array(data)
    data = data.reshape(samples,subseq,75)
    if version == 2:
        data = np.delete(data,[0,2,3,4,5,7,8,10,11,12,14],1)
        

    #label
    label = [int(x) for x in (label)]     
    label=np.asarray(label)
    #print (data.shape, label.shape)
    
    f.close()

    return data,label

In [28]:
def load_dataset(data_path, flabel_path,subseq, version):
    data = []
    label = []
    i=0
    for filename in os.listdir(data_path):
        data1, label1 = load_file(os.path.join(data_path,filename),os.path.join(flabel_path,filename),subseq, version)
        data.append(torch.tensor(data1).float())
        label.append(torch.tensor(label1))
        i+=1
        
    data= torch.cat(data, 0)
    label= torch.cat(label, 0)
    print ('nb file = ', i)
    
    #print (data.shape)
    #print (label.shape)
    
    return data,label

In [29]:
class MyDataset(Dataset):
    def __init__(self, data_path, flabel_path, subseq, version):
        #data loading
        data, label = load_dataset(data_path, flabel_path, subseq, version)
        self.x = data
        self.y = label
        self.n_samples = data.shape[0]
        
        
    def __getitem__(self, index):
        #dataset[0]
        return self.x[index], self.y [index]
        
    def __len__(self):
        #len dataset
        return self.n_samples

In [30]:
#load train data
subseq = 15
version = 2
data_path= '/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_train'
flabel_path='/home/oussema/code/st-gcn/pku/pku_action_detect/train'

datapku_train = MyDataset(data_path, flabel_path, subseq, version)
dataloader_train = DataLoader(dataset=datapku_train, batch_size = 128, drop_last=True)

nb file =  928


In [31]:
#load test data
subseq = 15
version = 2
data_path= '/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/sub_test'
flabel_path='/home/oussema/code/st-gcn/pku/pku_action_detect/test'
datapku_test = MyDataset(data_path, flabel_path, subseq, version )
dataloader_test = DataLoader(dataset=datapku_test, batch_size = 128, drop_last=True)

nb file =  130


In [20]:
#load exemple data
subseq = 15
version = 2
data_path= '/home/oussema/code/PKU-MMD/PKU_Skeleton_Renew/data/exemple'
flabel_path='/home/oussema/code/st-gcn/pku/pku_action_detect/train'

datapku_exemple = MyDataset(data_path, flabel_path, subseq, version)
dataloader_exemple = DataLoader(dataset=datapku_exemple, batch_size = 2, drop_last=True)

nb file =  12


In [32]:
#afficher data
dataiter = iter(dataloader_test)
data = dataiter.next()
x,y = data

In [33]:
print(x.shape)
len(datapku_test)

torch.Size([128, 4, 75])


40817

In [34]:
#many to one RNN-LSTM

class LSTMModel(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, batch_first=True):
        ''' Initialize the layers of this model.'''
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_dim


        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, 1) #output size = 1
        self.sig = nn.Sigmoid()

        
        # initialize the hidden state (see code below)
        #self.hidden = self.init_hidden()

        
    def init_hidden(self):
        
        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        return (torch.zeros(1, 4, self.hidden_dim).cuda(),
                torch.zeros(1, 4, self.hidden_dim).cuda())
    
    

    def forward(self, x, hidden):
        ''' Define the feedforward behavior of the model.'''
        
        training_batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x, hidden)#[1,4,2]    
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) #[1*4,2]
        out = self.fc(lstm_out)#[4,1]
        sig_out = self.sig(out) #[4,1]
        
        sig_out = sig_out.view(training_batch_size, -1)

        sig_out = sig_out[:, -1] # get last batch of labels

        
        #derniere hiden value
        #tag_outputs = tag_outputs[:,6:]
        
        #tag_scores = F.log_softmax(tag_outputs, dim=1)
        
        return  hidden, sig_out

In [35]:
EMBEDDING_DIM = 75
HIDDEN_DIM = 16

model = LSTMModel(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True)

#criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
criterion = nn.BCELoss()

optimizer = optim.SGD(model.parameters(), lr=0.1)

In [36]:
model = model.to('cuda:0')
model

LSTMModel(
  (lstm): LSTM(75, 16)
  (fc): Linear(in_features=16, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [37]:
for m in model.parameters():
    print(m.device) #return cuda:0

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [ ]:
epochs = 10
clip=5

for epoch in range (epochs):
    correct = 0
    train_losses = []
    print(epoch)
    hidden = model.init_hidden()
    
    model.train()
    for batch_idx, (data,label) in enumerate(dataloader_train):
        if cuda_available:
                data, label = data.cuda(), label.cuda()
                
        optimizer.zero_grad()
                
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        hidden= tuple([each.data for each in hidden])
                
        hidden, output = model(data.float(),hidden)


        loss = criterion (output , label.float())
        loss.backward() #retain_graph=True
        nn.utils.clip_grad_norm_(model.parameters(), clip) #prevent exploding gradients
        optimizer.step()
        
        pred = torch.round(output.squeeze())
        correct += pred.eq(label).sum().item()
        train_losses.append(loss.item())
        
        #print(pred,'******',label)
        
        
        
    train_acc = 100*correct/len(datapku_train)
    print(epoch," : ",train_acc)
    print(statistics.mean(train_losses))

0
0  :  69.96302588188269
0.5678865454058403
1
1  :  77.2415975483828
0.4880034649740658
2
2  :  78.88244895239998
0.46512436834670334
3
3  :  80.07727923786683
0.4495545879610058
4
4  :  80.83341660837414
0.4395361940140155
5
5  :  81.32540554944872
0.43245636576147223
6


In [ ]:
# run a test loop
test_loss = 0
correct = 0


model.eval()
hidden = model.init_hidden()

with torch.no_grad():  
    for batch_idx, (data,label) in enumerate(dataloader_train):
        if cuda_available:
                data, label = data.cuda(), label.cuda()
                
        hidden= tuple([each.data for each in hidden])
        
        hidden,net_out = model(data,hidden)
        # sum up batch loss
        test_loss += criterion(net_out, label.float()).data
        pred = torch.round(output.squeeze())  # get the index of the max log-probability
        correct += pred.eq(label.data).sum().item()

        
    test_acc = 100*correct/len(datapku_train)
    print (test_acc)